In [9]:
import requests
import numpy as np
import pandas as pd
import json
import datetime

In [11]:
with open('../data/apikeys.json') as fi:
    credentials = json.load(fi)
api_key = credentials['noaa']

In [13]:
# Next step: try collecting data from 1/1/2018-3/31/2025 !

In [15]:
# Try from this url:
url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-01&endDate=2025-03-31&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json'
# If that doesn't work, work on looping through all the dates and requesting batches!

In [37]:
# Note: the date range is hard coded into the url for the test run.

In [17]:
headers = {"token":api_key}

In [ ]:
r = requests.get(url, headers = headers)
print(r)

In [23]:
res = r.json()

In [41]:
print(len(res))
print(res[0])

1917
{'DATE': '2020-01-01', 'STATION': 'USW00013897', 'TMAX': '52', 'TMIN': '33', 'PRCP': '0.00'}


In [57]:
noaa_data = pd.DataFrame(columns = ['date','tmax','tmin','prcp'])
for weather_data in res :
    try :                                              # apparently the data from 2020-12-30 & 31 don't return PRCP values.
        new_row = {'date': weather_data['DATE'],       # I added in this try / except block to handle the missing prcp numbers within this loop.
                   'tmax': weather_data['TMAX'],
                   'tmin': weather_data['TMIN'],
                   'prcp': weather_data['PRCP']}
    except :
        new_row = {'date': weather_data['DATE'],
                   'tmax': weather_data['TMAX'],
                   'tmin': weather_data['TMIN'],
                   'prcp': '0.00'}
        print('error in this line:')
        print(weather_data)
    noaa_data.loc[len(noaa_data)] = new_row

error in this line:
{'DATE': '2020-12-30', 'STATION': 'USW00013897', 'TMAX': '68', 'TMIN': '38'}
error in this line:
{'DATE': '2020-12-31', 'STATION': 'USW00013897', 'TMAX': '45', 'TMIN': '41'}


In [59]:
noaa_data.to_csv('../data/noaa_weather_data.csv', index=False)

In [43]:
# If I need to request in batches, this is how to get a list of all the dates... and urls...
# update : I do not need to request in batches, the API was able to deliver the full set of data from 2020-01-01 to 2025-03-31 in one url request

In [35]:
startdate = datetime.date(2018,1,1)
enddate = datetime.date(2025,3,31)

# if I go this route, add a sleep timer to avoid bombarding their servers too much!
# and maybe modify this code to request in batches of like 1 month or 10 days at a time, rather than just 1 at a time?
# depends on how it goes trying to request the full 5-year-3-month chunk...

i = 1

requestdate = startdate
while requestdate <= enddate :
    url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate='
    url +=  str(requestdate)
    url += '&endDate='
    url += str(requestdate)
    url += '&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json'
    if i <= 5 :
        print(url)
    requestdate += datetime.timedelta(days=1)
    i += 1

print('... and so on...')
print('final url: ' + url)

https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-01&endDate=2020-01-01&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-02&endDate=2020-01-02&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-03&endDate=2020-01-03&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-04&endDate=2020-01-04&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-05&endDate=2020-01-05&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
... and so on...
final url: https://www.ncei.noaa.gov/access/serv